In [1]:
import importlib as imp

In [2]:
# Utilities
def verbose_time():
    import time
    from datetime import datetime
    utc_ts = int(time.time())
    time_str = datetime.fromtimestamp(utc_ts).strftime('%Y-%m-%d %H:%M:%S')
    print(time_str)
verbose_time()

2019-08-16 11:10:24


In [3]:
# Process data
FN_ITEM_TITLE = "../data/item_title.index"
FN_UI_ADJ = "../data/user_items.adjlist"
FN_CORPUS = "../data/random_walks.corpus"

In [4]:
def load_item_titles(fn):
    """
        Load item titles
    """
    item_title_dict = {}
    idx = 0
    with open(fn) as fd:
        for line in fd:
            line = line.rstrip()
            item_title_dict["I{}".format(idx)] = line
            idx += 1
    return item_title_dict

def readable_results(entries, item_title_dict):
    """
        Print readable search results
    """
    for entry in entries:
        if entry[0] in item_title_dict:
            print("{} {} {}".format(entry[0], entry[1], item_title_dict[entry[0]]))
        else:
            print("{} {}".format(entry[0], entry[1]))

item_title_dict = load_item_titles(FN_ITEM_TITLE)
print("I0: {}".format(item_title_dict["I1"]))

I0: 动画片 | 超级飞侠第6季：超级飞侠 勇闯天下


In [5]:
def load_user_items_dict(fn):
    """
        Load user-items adjacent dict
    """
    user_items_dict = {}
    with open(fn) as fd:
        for line in fd:
            arr = line.rstrip().split(" ")
            if len(arr) < 2:
                continue
            user_items_dict[arr[0]] = arr[1:]
    return user_items_dict

def readable_user_items(user, user_items_dict, item_title_dict):
    """
        Print readable user's item history
    """
    for item in user_items_dict[user]:
        print("{} {}".format(item, item_title_dict[item]))

user_items_dict = load_user_items_dict(FN_UI_ADJ)
print("U1->items: {}".format(user_items_dict["U5670"]))
readable_user_items("U5670", user_items_dict=user_items_dict, item_title_dict=item_title_dict)

U1->items: ['I24', 'I4', 'I7', 'I1', 'I180', 'I27']
I24 电视剧 | 少年江湖物语：鲜衣怒马 青春江湖
I4 动画片 | 汪汪队立大功第4季：本领高强的狗狗巡逻队
I7 电视剧 | 蜜汁炖鱿鱼：杨紫李现暖心爱恋
I1 动画片 | 超级飞侠第6季：超级飞侠 勇闯天下
I180 为什么，问小度丨小度小度，为什么会有肉食植物
I27 歌单推荐丨励志男声 · 倔强追梦路


In [6]:
# Create graph
import networkx as nx
def create_unweighted_graph_with_adjlist_format(fn, is_directed=False):
    '''
        Creates an undirected unweighted graph with adjacent list format file
    '''
    G = nx.read_adjlist(fn, comments='#', delimiter=" ", nodetype=str)
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1
    if is_directed:
        G = G.to_directed()
    return G
G = create_unweighted_graph_with_adjlist_format(FN_UI_ADJ, is_directed=False)

In [7]:
import random
random.sample(list(G.neighbors("U5670")), 1)

['I24']

In [8]:
# Create walker
import graphwalker
graphwalker = imp.reload(graphwalker)
verbose_time()
walker = graphwalker.GraphWalker(G, p=1, q=2)
verbose_time()

2019-08-16 11:10:27
processed 10000/84018 nodes.
processed 20000/84018 nodes.
processed 30000/84018 nodes.
processed 40000/84018 nodes.
processed 50000/84018 nodes.
processed 60000/84018 nodes.
processed 70000/84018 nodes.
processed 80000/84018 nodes.
processed 10000/295576 edges.
processed 20000/295576 edges.
processed 30000/295576 edges.
processed 40000/295576 edges.
processed 50000/295576 edges.
processed 60000/295576 edges.
processed 70000/295576 edges.
processed 80000/295576 edges.
processed 90000/295576 edges.
processed 100000/295576 edges.
processed 110000/295576 edges.
processed 120000/295576 edges.
processed 130000/295576 edges.
processed 140000/295576 edges.
processed 150000/295576 edges.
processed 160000/295576 edges.
processed 170000/295576 edges.
processed 180000/295576 edges.
processed 190000/295576 edges.
processed 200000/295576 edges.
processed 210000/295576 edges.
processed 220000/295576 edges.
processed 230000/295576 edges.
processed 240000/295576 edges.
processed 250

In [9]:
def sample_walks(iterator, num_samples):
    arr = []
    for _ in range(num_samples):
        item = next(iterator, None)
        if item is None:
            return arr
        arr.append(item)
    return arr

samples = sample_walks(walker.simulate_walks(num_epochs=1, walk_len=10), num_samples=5)
print(samples)

epoch: 1/1
[['U41119', 'I1', 'U81988', 'I1', 'U55262', 'I1', 'U70558', 'I2', 'U65954', 'I2'], ['U79793', 'I68', 'U9492', 'I56', 'U2813', 'I122', 'U29883', 'I56', 'U19033', 'I9'], ['U84337', 'I90', 'U52912', 'I128', 'U71213', 'I2', 'U51752', 'I2', 'U23613', 'I146'], ['U97586', 'I59', 'U44804', 'I59', 'U41840', 'I59', 'U34307', 'I59', 'U3489', 'I72'], ['U49660', 'I8', 'U47701', 'I8', 'U48066', 'I92', 'U5639', 'I2', 'U10043', 'I0']]


In [10]:
# Generate corpus
def generate_corpus(generator, fn_corpus):
    """
        Generate corpus
    """
    with open(fn_corpus, "w") as fd:
        for walk in generator:
            fd.write("{}\n".format(" ".join(walk)))
verbose_time()
generate_corpus(walker.simulate_walks(num_epochs=5, walk_len=10), FN_CORPUS)
verbose_time()

2019-08-16 12:20:45
epoch: 1/5
 processed: 10000/84018
 processed: 20000/84018
 processed: 30000/84018
 processed: 40000/84018
 processed: 50000/84018
 processed: 60000/84018
 processed: 70000/84018
 processed: 80000/84018
epoch: 2/5
 processed: 10000/84018
 processed: 20000/84018
 processed: 30000/84018
 processed: 40000/84018
 processed: 50000/84018
 processed: 60000/84018
 processed: 70000/84018
 processed: 80000/84018
epoch: 3/5
 processed: 10000/84018
 processed: 20000/84018
 processed: 30000/84018
 processed: 40000/84018
 processed: 50000/84018
 processed: 60000/84018
 processed: 70000/84018
 processed: 80000/84018
epoch: 4/5
 processed: 10000/84018
 processed: 20000/84018
 processed: 30000/84018
 processed: 40000/84018
 processed: 50000/84018
 processed: 60000/84018
 processed: 70000/84018
 processed: 80000/84018
epoch: 5/5
 processed: 10000/84018
 processed: 20000/84018
 processed: 30000/84018
 processed: 40000/84018
 processed: 50000/84018
 processed: 60000/84018
 processed: 7

In [11]:
# Learn embeddings
import word2vec as w2v
w2v = imp.reload(w2v)
verbose_time()
model = w2v.learn(FN_CORPUS, num_dims=32, window_size=5)
verbose_time()

2019-08-16 13:07:47


/anaconda3/envs/tf13/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-08-16 13:08:26


In [17]:
# test
w2v = imp.reload(w2v)
query = "U361"
entries = w2v.search(model.wv, [query], topk=100000)
filtered_entries = []
for entry in entries:
    if entry[0][0] == "I":
        filtered_entries.append(entry)
#print("IN: {} {}".format(query, item_title_dict[query]))
readable_results(filtered_entries[:20], item_title_dict=item_title_dict)

I216 0.6562332510948181 歌单推荐丨清凉一夏：  畅游电子泳池
I933 0.6192339062690735 听风水｜张赟慧说风水：不可不知的办公室风水
I262 0.5789454579353333 电影 | 惊奇队长：揭秘最强英雄起源
I1062 0.575980544090271 音乐MV丨为你推荐田馥甄的《你就不要想起我》
I55 0.560906171798706 歌单推荐丨文雅流行，温润的浪漫年代
I391 0.5515677332878113 动漫 | 梦幻书院：梦幻西游手游同人动画
I448 0.549060583114624 电视剧 | 青春斗：郑爽领衔演绎奋斗青春
I296 0.5455194711685181 纪录片 | 大国质量：企业版“大国崛起”
I658 0.5414429903030396 电影 | 飞驰人生：沈腾黄景瑜爆燃对决
I293 0.5355770587921143 电影 | 雷霆沙赞：搞怪英雄前来参见
I764 0.5351083278656006 音乐MV丨为你推荐吴亦凡的《大碗宽面》
I64 0.5340047478675842 歌单推荐丨深夜音“药”： 激活你的学习脑细胞
I31 0.5333694815635681 歌单推荐丨地下暗黑势力 · 狂躁日语Rap
I145 0.5299396514892578 听知识｜超级思维：怎样成为很厉害的人
I17 0.528771162033081 动漫 | 龙珠超布罗利：悟吉塔大战布罗利
I240 0.5244192481040955 试着说：小度小度，我想听熊猫天天讲故事
I350 0.5218261480331421 垃圾分类丨铲屎的，狗毛你知道是什么垃圾嘛？
I161 0.5181187987327576 是真是假，问小度丨小度小度，保鲜膜可以减肥吗
I328 0.5179069638252258 歌单推荐丨地名民谣 · 致那些漂泊的岁月
I144 0.515987753868103 为什么，问小度丨小度小度，为什么蝎毒如此厉害


In [13]:
readable_user_items("U361", user_items_dict=user_items_dict, item_title_dict=item_title_dict)

I24 电视剧 | 少年江湖物语：鲜衣怒马 青春江湖
I34 小度小度，播放杨宗纬的《是缘》
I55 歌单推荐丨文雅流行，温润的浪漫年代
I56 垃圾分类丨爆米花一定要吃完，不然真的会很麻烦
I98 歌单推荐丨民谣里的姑娘，总是温柔又哀伤
I7 电视剧 | 蜜汁炖鱿鱼：杨紫李现暖心爱恋
I47 小度小度，播放刺猬乐队的《24小时摇滚聚会》
I45 综艺 | 中国新说唱2019：吴亦凡“抛弃”黄旭
I40 歌单推荐丨伤感情歌 · 爱是一场高烧
I27 歌单推荐丨励志男声 · 倔强追梦路
I3 动画片 | 小猪佩奇第6季：佩奇一家的精彩生活
I151 垃圾分类丨牙齿本来就不能乱扔。
I6 工作了一整天，回家终于可以做回自己
I84 情感电台｜幸福女人的聪明恋爱术
I18 听一首歌放松一下吧，工作不着急一时完成
I85 歌单推荐丨韩式小清新 : 解锁恋爱新格调
I15 综艺 | Vlog营业中：揭秘明星工作台前幕后
I180 为什么，问小度丨小度小度，为什么会有肉食植物
I157 小度小度，播放吴亦凡的《大碗宽面》
I59 歌单推荐丨日式消暑：炎夏必备清凉曲
I91 歌单推荐丨韩系女友嗓 : 治愈心灵小调
I0 动画片 | 海绵宝宝：剧情幽默而充满想象力
I38 小度小度，播放薛之谦的《慢半拍》
I2 动画片 | 喜羊羊与灰太狼之羊村守护者：羊狼发明大作战
I62 小度小度，播放邓超的《银河里最像的人》
I78 电视剧 | 追球：巧克力吻甜蜜升级
I26 小度小度，播放摩登兄弟的《如约》
I104 查百科，问小度丨小度小度，黑洞会发光吗
I83 歌单推荐丨效率提速器 · 打败拖延症
I53 小度小度，播放许嵩的歌
I52 音乐MV丨为你推荐周杰伦的《雨下一整晚》
I10 电视剧 | 请赐我一双翅膀：鞠婧祎炎亚纶历磨难
I166 垃圾分类丨你猜方便面调料包算什么垃圾？
I31 歌单推荐丨地下暗黑势力 · 狂躁日语Rap
I144 为什么，问小度丨小度小度，为什么蝎毒如此厉害
I1 动画片 | 超级飞侠第6季：超级飞侠 勇闯天下
I117 怎么办，问小度丨小度小度，宝宝有痰怎么办
I116 垃圾分类丨每天吃鸡蛋，但这个尝试要指导！
I216 歌单推荐丨清凉一夏：  畅游电子泳池
I70 电影 | 企鹅公路：企鹅军团拯救世界
I87 歌单推荐丨华语流行：邂逅晚虹般的你
I33 

In [14]:
# Save
w2v = imp.reload(w2v)
w2v.save(model, "../output/user_item.emb", "../output/user_item.vocab")

In [15]:
# Load
w2v = imp.reload(w2v)
wv = w2v.load("../output/user_item.emb")